In [6]:
import pandas as pd
import matplotlib.pyplot as plt

# Lire le fichier depuis l'environnement Colab
df = pd.read_excel("hotels_avec_rate.xlsx")

# Afficher un aperçu des données
df.head()





,name,price,formule,region,date,rate
0,Hasdrubal Prestige Thalassa & SPA Djerba,151.417,Logement Simple,Djerba,2025-12-04,8.90
1,Le Beau Séjour,160.230,Logement Petit Déjeuner,Djerba,2025-03-25,6.80
2,hôtel club palm azur,173.964,Logement Petit Déjeuner,Djerba,2024-12-14,7.48
3,Club Calimera Yati Beach,206.010,Logement Petit Déjeuner,Djerba,2025-05-11,7.48
4,Joya Paradise,215.125,560.698 DT,Djerba,2024-11-25,7.48


In [7]:
import pandas as pd

# Affiche le nombre de valeurs NaN par colonne
df.isna().sum()


,0
name,0
price,2
formule,0
region,0
date,0
rate,0


In [8]:

df['formule'].unique()
import pandas as pd
import numpy as np

# 1. Nettoyage des espaces
df['formule'] = df['formule'].str.strip()

# 2. Fonction de détection de valeur "corrompue"
def est_prix(val):
    if isinstance(val, str):
        return bool(pd.Series(val).str.contains(r'\d+[,\.]?\d*\s*DT').any())
    return False

# 3. Remplacer les valeurs qui sont des prix par NaN
df['formule'] = df['formule'].apply(lambda x: np.nan if est_prix(x) else x)

# 4. Afficher les valeurs uniques restantes

df['formule'].fillna(df['formule'].mode()[0], inplace=True)
print(df['formule'].unique())


['Logement Simple' 'Logement Petit Déjeuner' 'Soft All Inclusive'
 'Demi Pension' 'All Inclusive' 'All Inclusive GOLD']


<ipython-input-8-70851b29b378>:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['formule'].fillna(df['formule'].mode()[0], inplace=True)


In [19]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt

# 1. Préparation des données
# Assurez-vous que la région est encodée si elle est catégorielle
# Si 'region' est une colonne catégorielle, utilisez pd.factorize pour l'encoder en numérique
# Nettoyage des noms de région
# Nettoyage des noms de région
corrections = {
    'hamamet': 'Hammamet',
    'hammet': 'Hammamet',
    'hammamet': 'Hammamet',
    'djerba': 'Djerba',
    'sousse': 'Sousse',
    'monastir': 'Monastir',
}

df['region'] = df['region'].str.lower().str.strip()
df['region'] = df['region'].replace(corrections)
df['region'] = df['region'].str.title()  # Pour avoir une belle mise en forme

# Encodage
region_encoder = LabelEncoder()
df['region_encoded'] = region_encoder.fit_transform(df['region'])
# Sélectionner uniquement les colonnes 'region_encoded' et 'rate'
data = df[['region_encoded', 'rate']].values

# 2. Clustering DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
labels_dbscan = dbscan.fit_predict(data)

# 3. Post-traitement des clusters
n_clusters = len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0)  # Exclure les points de bruit (-1)
noise_percentage = (list(labels_dbscan).count(-1) / len(labels_dbscan)) * 100  # Pourcentage de bruit

# 4. Calcul des métriques
valid_mask = labels_dbscan != -1
if sum(valid_mask) > 1 and n_clusters > 1:
    silhouette = silhouette_score(data[valid_mask], labels_dbscan[valid_mask])
    db_score = davies_bouldin_score(data[valid_mask], labels_dbscan[valid_mask])
else:
    silhouette, db_score = float('nan'), float('nan')

# 6. Analyse des résultats
print(f"""
=== Résultats DBSCAN ===
- Clusters identifiés : {n_clusters}
- Points de bruit : {noise_percentage:.1f}%
- Score de Silhouette : {silhouette:.2f} (cible >0.5)
- Indice Davies-Bouldin : {db_score:.2f} (cible <1)



""")

# 7. Export des résultats (si DataFrame disponible)
df['cluster'] = labels_dbscan
cluster_stats = df.groupby('cluster').agg({
    'rate': 'mean',
    'region': lambda x: x.mode()[0],  # La région la plus fréquente dans chaque cluster
}).reset_index()

print("\nStatistiques par cluster :")
print(cluster_stats)



=== Résultats DBSCAN ===
- Clusters identifiés : 6
- Points de bruit : 10.7%
- Score de Silhouette : 0.91 (cible >0.5)
- Indice Davies-Bouldin : 0.22 (cible <1)





Statistiques par cluster :
   cluster      rate    region
0       -1  7.215789    Sousse
1        0  7.478571    Djerba
2        1  7.480000  Hammamet
3        2  8.966667  Hammamet
4        3  5.600000  Hammamet
5        4  7.470000    Sousse
6        5  7.473793  Monastir


In [41]:
corrections = {
    'hamamet': 'Hammamet',
    'hammet': 'Hammamet',
    'hammamet': 'Hammamet',
    'djerba': 'Djerba',
    'sousse': 'Sousse',
    'monastir': 'Monastir',
}
# Tout en minuscules, strip, puis correction
df['region'] = df['region'].str.lower().str.strip()
df['region'] = df['region'].replace(corrections)


# Nettoyage des noms de région
df['region'] = df['region'].str.strip().str.capitalize()

# Encodage
region_encoder = LabelEncoder()
df['region_encoded'] = region_encoder.fit_transform(df['region'])


print("\n=== Codage des régions ===")
for region, code in zip(region_encoder.classes_, region_encoder.transform(region_encoder.classes_)):
    print(f"{region} → {code}")



=== Codage des régions ===
Djerba → 0
Hammamet → 1
Monastir → 2
Sousse → 3


In [14]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# 1. Préparation des données
# Nettoyage des noms de région
corrections = {
    'hamamet': 'Hammamet',
    'hammet': 'Hammamet',
    'hammamet': 'Hammamet',
    'djerba': 'Djerba',
    'sousse': 'Sousse',
    'monastir': 'Monastir',
}

df['region'] = df['region'].str.lower().str.strip()
df['region'] = df['region'].replace(corrections)
df['region'] = df['region'].str.title()  # Pour avoir une belle mise en forme

# Encodage
region_encoder = LabelEncoder()
df['region_encoded'] = region_encoder.fit_transform(df['region'])

# === Affichage du codage des régions ===
print("\n=== Codage des régions ===")
for region, code in zip(region_encoder.classes_, region_encoder.transform(region_encoder.classes_)):
    print(f"{region} → {code}")

# 2. Données pour le clustering
data = df[['region_encoded', 'rate']].values

# 3. Détermination du nombre optimal de clusters (méthode du coude)
wcss = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data)
    wcss.append(kmeans.inertia_)

# 4. Clustering K-Means (K=4 par exemple)
k = 4
kmeans = KMeans(n_clusters=k, random_state=42)
labels_kmeans = kmeans.fit_predict(data)

# 5. Calcul des métriques
silhouette = silhouette_score(data, labels_kmeans)
db_score = davies_bouldin_score(data, labels_kmeans)

# 6. Interprétation des clusters
df['cluster'] = labels_kmeans
cluster_stats = df.groupby('cluster').agg({
    'rate': ['mean', 'min', 'max'],
    'region': lambda x: x.mode()[0]
}).reset_index()

print("\n=== Statistiques par cluster ===")
print(cluster_stats)

print(f"""
=== Résultats K-Means ===
- Nombre de clusters : {k}
- Score de Silhouette : {silhouette:.2f} (cible >0.5)
- Indice Davies-Bouldin : {db_score:.2f} (cible <1)

=== Interprétation recommandée ===
luster 0 : Hôtels milieu de gamme avec des notes modérées à élevées

Cluster 1 : Hôtels de gamme supérieure, de qualité plus élevée

Cluster 2 : Hôtels haut de gamme ou de luxe, avec des prestations plus poussées

Cluster 3 : Hôtels économiques, de qualité plus basse avec des prix abordables
""")



=== Codage des régions ===
Djerba → 0
Hammamet → 1
Monastir → 2
Sousse → 3

=== Statistiques par cluster ===
  cluster      rate              region
               mean  min  max  <lambda>
0       0  7.678182  6.3  9.9    Sousse
1       1  7.733412  6.9  9.8  Hammamet
2       2  7.552273  6.8  9.3    Djerba
3       3  5.518750  5.0  5.9  Hammamet

=== Résultats K-Means ===
- Nombre de clusters : 4
- Score de Silhouette : 0.54 (cible >0.5)
- Indice Davies-Bouldin : 0.66 (cible <1)

=== Interprétation recommandée ===
luster 0 : Hôtels milieu de gamme avec des notes modérées à élevées

Cluster 1 : Hôtels de gamme supérieure, de qualité plus élevée 

Cluster 2 : Hôtels haut de gamme ou de luxe, avec des prestations plus poussées 

Cluster 3 : Hôtels économiques, de qualité plus basse avec des prix abordables 



In [18]:
# Fonction de prédiction pour un nouvel hôtel
def predict_cluster(region, rate):
    region = region.lower().strip().title()
    if region in region_encoder.classes_:
        region_encoded = region_encoder.transform([region])[0]
        data_point = np.array([[region_encoded, rate]])
        return kmeans.predict(data_point)[0]
    else:
        return "Région inconnue"

# Exemple de prédictions pour de nouveaux hôtels
test_cases = [
    {"region": "Monastir", "rate": 3},

    {"region": "Djerba", "rate": 2},
     # À ajouter si elle existe dans tes données
]

print("\n=== Prédictions pour de nouveaux hôtels ===")
for hotel in test_cases:
    cluster = predict_cluster(hotel["region"], hotel["rate"])
    print(f"Région: {hotel['region']}, Note: {hotel['rate']} → Cluster {cluster}")



=== Prédictions pour de nouveaux hôtels ===
Région: Monastir, Note: 3 → Cluster 3
Région: Djerba, Note: 2 → Cluster 3


In [20]:
# Fonction de prédiction pour un nouvel hôtel
def predict_cluster(region, rate):
    region = region.lower().strip().title()
    if region in region_encoder.classes_:
        region_encoded = region_encoder.transform([region])[0]
        data_point = np.array([[region_encoded, rate]])
        return kmeans.predict(data_point)[0]
    else:
        return "Région inconnue"

# Exemple de prédictions pour de nouveaux hôtels
test_cases = [
    {"name": "Palm Beach Monastir", "region": "Monastir", "rate": 3},
    {"name": "Djerba Resort", "region": "Djerba", "rate": 2},
]

print("\n=== Prédictions pour de nouveaux hôtels ===")
for hotel in test_cases:
    cluster = predict_cluster(hotel["region"], hotel["rate"])

    if isinstance(cluster, int):  # Vérifie que c'est un cluster valide
        hotels_in_cluster = df[df["cluster"] == cluster]["hotel_name"].unique()
        print(f"Hôtel: {hotel['name']}, Région: {hotel['region']}, Note: {hotel['rate']} → Cluster {cluster}")
        print(f"Hôtels existants dans le Cluster {cluster} :")
        for h in hotels_in_cluster:
            print(f" - {h}")
        print()
    else:
        print(f"Hôtel: {hotel['name']} → {cluster}")



=== Prédictions pour de nouveaux hôtels ===
Hôtel: Palm Beach Monastir → 3
Hôtel: Djerba Resort → 3
